# Demo Usage of the Generalizer (GENERALization-optimiZER)

The Generalizer optimizes faster by maintaining a n+1 dimensional surrogate model which incorporates the samples of already solved similar problems with shared features. The additional axis is not vital to solve each optimization problem but connects similar problems from the same class. We thus speak from generalizing between optimization problems. We call the n dimensional problems (hyper)slices of the n+1 dimensional surrogate space (intuition: 1D slices of a 2D problem space).

As example function to be optimized we use the Six Hump Camel function. We evaluate slices along the first dimension, the second dimension forms the additional axis which connects related problems in this class.

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import sys
from generalizer import Generalizer
import demo_visualization
import utils
%matplotlib inline

*Exemplary convergence criterion:*

The minimum in y direction was not changed significantly in the last rounds as well as the x position did not change more than a tolerance.

In [2]:
# variables needed to track convergence
y_min = sys.float_info.max
slice_minimas = np.ones(3) * 1e20
last_x = np.array([[-1.0, 0.0],[0.0, -1.0]])


def converged(slice_minimas, y_min, last_x):
    limit = 3
    tol = 0.02
    y_conv = 0 <= slice_minimas[-limit]/y_min - 1 < tol
    x_conv = np.allclose(last_x[-1], last_x[-2], atol=tol)
        
    return y_conv and x_conv

The Generalizer is initialized with data assumed to belong to previous related experiments. Based upon this data a new slice of the surrogate model is evaluated.

In the following the setup is described. Much goes into setting up and scaling the artificial example samples.

In [3]:
# function space
dim = np.array([[-2, -1], [2, 1]])

# quantization in each dimension
grid_size = 101

# for demo purposes we initialized some slices already (in deployment that will be done with existing data)
# the chosen slices must lie within the grid spacing and the final samples be transformed to range [0,1]
initial_slices_single = np.random.choice(np.linspace(dim[0][1], dim[1][1], grid_size), 5, replace=False)
initial_slices = np.repeat(initial_slices_single, 10)
initial_spacing = np.tile(np.linspace(dim[0][0], dim[1][0], 10), 5)
initial_samples_X = np.vstack((initial_spacing, initial_slices)).T
initial_samples_Y = utils.six_hump_camel(initial_samples_X)

# scaling to [0,1]
initial_samples_X = utils.scale_01(initial_samples_X, dim)
initial_slices_single = (initial_slices_single+1.0) / 2.0



# instantiate generalizer with surrogate
print('>>>>>> Randomly initialize with slices: ', initial_slices_single)

generalizer = Generalizer(dim,
                          initial_samples_X=initial_samples_X,
                          initial_samples_Y=initial_samples_Y,
                          grid_size=grid_size)

('>>>>>> Randomly initialize with slices: ', array([ 0.38,  0.27,  0.61,  0.07,  0.97]))


In the main loop new sampling locations are seeked for and then evaluated. It is the users task to acquire the required samples by experiment or simulation. We simulate this by evaluating the provided Six Hump Camel function.

In [4]:
# loop to optimize current slice
current_slice = np.random.choice(np.linspace(0, 1, grid_size))
print('>>>>>> Optimize slice: ', current_slice)
i = 0

while not converged(slice_minimas, y_min, last_x):
    print('>>>>>>   Sample ', i)

    # get next evaluation location
    new_x, proof = generalizer.get_next_sampling_location(current_slice)

    # evaluate six hump to get new sample
    original_scale_new_x = utils.scale_restore(np.matrix(new_x), dim)
    new_y = utils.six_hump_camel(np.array(original_scale_new_x).squeeze())

    # incorporate into surrogate model
    generalizer.incorporate_new_sample(new_x, new_y)

    # update convergence variables
    slice_samples_X, slice_samples_Y = generalizer.get_samples_for(current_slice)
    y_min = np.min(slice_samples_Y)
    slice_minimas = np.append(slice_minimas, y_min)
    last_x = np.vstack((last_x, new_x))

    # visualize results
    demo_visualization.plot(proof, current_slice, grid_size, dim, slice_samples_X, slice_samples_Y, 
                            initial_slices_single, initial_samples_X, initial_samples_Y)

    i += 1


print('>>>>>> FINISHED! Samples until convergence: ', i)
print('>>>>>> Minimum %f at position '%y_min, last_x[-1])

('>>>>>> Optimize slice: ', 0.48999999999999999)
('>>>>>>   Sample ', 0)
Neurotic Natalie tamed - Training error=0.001264079
Neurotic Natalie tamed - Training error=0.021809166
Neurotic Natalie tamed - Training error=0.020493859
Neurotic Natalie tamed - Training error=0.008734651
Neurotic Natalie tamed - Training error=0.019320082
Neurotic Natalie tamed - Training error=0.002705889
Neurotic Natalie tamed - Training error=0.013732753
Neurotic Natalie tamed - Training error=0.002898110
Neurotic Natalie tamed - Training error=0.013927061
Neurotic Natalie tamed - Training error=0.009431352
Neurotic Natalie tamed - Training error=0.021760797
Neurotic Natalie tamed - Training error=0.004050029
Neurotic Natalie tamed - Training error=0.025245419
Neurotic Natalie tamed - Training error=0.016075065
Neurotic Natalie tamed - Training error=0.019802270
Neurotic Natalie tamed - Training error=0.007972329
Neurotic Natalie tamed - Training error=0.009558741
Neurotic Natalie tamed - Training error=0.0

ValueError: Unknown projection '3d'